# Introduction
For this part of the assignment, we are still going to use the FrozenLake-v0 environment. We are going to implment the Q learning algorithm and evaluate it on the Frozenlake environment. 

In [3]:
import gym
import time
import numpy
import matplotlib.pyplot as plt

def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

query_environment("FrozenLake-v1")

Action Space: Discrete(4)
Observation Space: Discrete(16)
Max Episode Steps: 100
Nondeterministic: False
Reward Range: (0, 1)
Reward Threshold: 0.7


# Q Learning Implmentation 

In it’s simplest implementation, Q-Learning is a table of values for every state (row) and action (column) possible in the environment. Within each cell of the table, we learn a value for how good it is to take a given action within a given state. In the case of the FrozenLake environment, we have 16 possible states (one for each block), and 4 possible actions (the four directions of movement), giving us a 16 x 4 table of Q-values. We start by initializing the table to be uniform (all zeros), and then as we observe the rewards we obtain for various actions, we update the table accordingly.

The update rule is 

Q(s,a) = (1 - lr) Q(s,a) + lr(r(s,a) + γ Q(s,a))

In [15]:
import numpy as np
import random
import gym
env = gym.make('FrozenLake-v1')

#Initialize table, with states as rows and actions (up, down, left, or right) as columns 
Q = np.zeros([env.observation_space.n, env.action_space.n])   # Q[nS][nA]
#Set learning parameters
lr = .8
#Set discounted factor
gamma = .95
epsilon = 0.05
num_episodes = 200000
#create lists to contain total rewards and steps per episode
# rewards = []
avg_reward = 0

for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()     # start_state
    #Total reward in one episode
    reward_tot = 0
    while True:
        ###############################################################################
        # TODO: Implement the Q-Table learning algorithm.                             #
        # You will need to do the following:                                          #
        # (1) Choose an action by greedily (with noise) picking from Q table given s  #
        #     as input.                                                               #
        # (2) Get new state s1, reward r and done d from environment                  #
        # (3) Update Q-Table with new knowledge.                                      #
        # (4) Cumulate the total reward                                           #
        # (5) Update s                                                                #
        # Note: You may use the gym interfaces env.action_space, env.step etc.        #
        #       E.g. observation, reward, done, info = env.step(action)               #
        #       Please refer to the docs for more information.                        #
        #       For (1), consider adding noise as a mean of encouraging exploration.  #
        #       For (3), calculate the new target Q-value using Bellman equation.     #
        #       Instead of directly updating toward it, we take a small step in the   #
        #       direction that will make the Q value closer to the target, i.e. use   #
        #       learning rate that controls how much of the difference between        #
        #       newly proposed Q-value and previous Q-value                           #
        ###############################################################################
        # greedy choice
        if numpy.random.choice([1, 0], p=[1-env.action_space.n*epsilon, env.action_space.n*epsilon]):
            action = numpy.argmax(Q[s])
        # random choice from [0,...,nA-1]
        else:
            action = random.choice([i for i in range(env.action_space.n)])
        next_state, reward, done, _ = env.step(action)
        Q[s][action] = Q[s][action] + lr*(reward + gamma*numpy.max(Q[next_state]) - Q[s][action])
        reward_tot += reward
        s = next_state
        ##############################################################################
        #                             END OF YOUR CODE                               #
        ##############################################################################

        #end of one episode
        if done == True:
            break
    avg_reward = (avg_reward*i+reward_tot)/(i+1)
    # print("Score over time: " +  str(avg_reward))
print(Q)
print(avg_reward)

[[2.21259560e-01 1.45610496e-01 3.36594249e-01 1.91829395e-01]
 [3.97124083e-04 4.04258718e-01 1.41985974e-01 1.28571209e-01]
 [1.06394234e-01 1.31860384e-01 4.18685897e-01 1.00766430e-01]
 [5.86185081e-03 7.02078869e-04 2.15528740e-03 1.02640021e-01]
 [2.36056218e-01 5.34777879e-02 2.13405888e-01 4.88682200e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.19179838e-02 1.34190002e-03 3.95423129e-01 1.29631716e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.31979566e-02 2.13127519e-01 1.13460566e-02 2.57087778e-01]
 [7.35351643e-02 4.63529547e-01 2.09151476e-02 1.50600730e-01]
 [3.87618146e-01 2.90688772e-02 6.66281963e-01 2.99059250e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.24427208e-01 1.20008108e-01 3.83995183e-01 1.06174017e-01]
 [5.75087453e-01 6.51544723e-01 9.39173849e-01 6.50681432e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000000